In [1]:
import pandas as pd
import os
from langchain_community.graphs import Neo4jGraph
from tqdm import tqdm
import numpy as np
import re


In [2]:
os.environ['NEO4J_URI'] = 'bolt://localhost:7690'
os.environ['NEO4J_USERNAME'] = 'neo4j'
os.environ['NEO4J_PASSWORD'] = 'Password@123'
graph = Neo4jGraph()

In [3]:
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

os.environ["AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_API_KEY")
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://sriks-openai.openai.azure.com/"
os.environ["AZURE_OPENAI_API_VERSION"] = "2024-05-01-preview"
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT"] = "gpt-4o"
os.environ["AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT"] = "text-embedding-ada-002"

llm = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT"],
)

embeddings = AzureOpenAIEmbeddings(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT"],
)

In [4]:
examples = [
    {
        "question": "What is the root cause request id 8ff8696695aa73588ac454809741e2ea",
        "query": "MATCH (n)-[r:DEPENDS_ON]->(m) where n.id <> 'ROOT' OR m.id <> 'ROOT' and r.operationId = 8ff8696695aa73588ac454809741e2ea RETURN n.id, r.duration, r.operation_Name, m.id ORDER BY r.duration DESC LIMIT 3",
    }
]

In [ ]:
from langchain.chains import GraphCypherQAChain
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_chroma import Chroma

graph.refresh_schema()

QA_GENERATION_TEMPLATE = """
       Task: answer the question you are given based on the context provided.
       Instructions:
        You are an assistant that helps to form nice and human understandable answers. 
        Use the context information provided to generate a well organized and comprehensve answer to the user's question. 
        When the provided information contains multiple elements, structure your answer as a bulleted or numbered list to enhance clarity and readability.
        You must use the information to construct your answer. 
        The provided information is authoritative; do not doubt it or try to use your internal knowledge to correct it. 
        Make the answer sound like a response to the question without mentioning that you based the result on the given information. 
        If there is no information provided, say that the knowledge base returned empty results.

        Here's the information:
        {context}

        Question: {question}
        Answer:
            """
EXAMPLES_PROMPT_TEMPLATE = """   
                Input: {question},
                Output: {query}
            """
qaPrompt = PromptTemplate(input_variables=["context", "question"], template=QA_GENERATION_TEMPLATE)
example_prompt = PromptTemplate(input_variables=["question", "query"], template=EXAMPLES_PROMPT_TEMPLATE)

cypherPromptTemplate = """Task:Generate Cypher statement to query a graph database.
Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
Schema:
{schema}

Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than for you to construct a Cypher statement.
Do not include any text except the generated Cypher statement.

The question is:
{question} 


"""

similaritySelector = SemanticSimilarityExampleSelector.from_examples(
    examples=examples, 
    embeddings=embeddings, 
    k=1,
    vectorstore_cls=Chroma
)

cypherPrompt = FewShotPromptTemplate(
    example_selector=similaritySelector,
    example_prompt=example_prompt,
    examples=examples,
    input_variables=["schema", "question"], 
    prefix=cypherPromptTemplate,
    suffix="The question is:\n{question}",
)

cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm = llm,
    qa_llm = llm, 
    return_intermediate_steps=True,
    validate_cypher=True,
    graph=graph, 
    verbose=True,
    qa_llm_kwargs={"prompt":qaPrompt},
    cypher_llm_kwargs={ "prompt": cypherPrompt},
)


: 

In [276]:
result = cypher_chain({"query": "What is the root cause request id 8ff8696695aa73588ac454809741e2ea"})
print(result['result'])



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (n)-[r:DEPENDS_ON]->(m) WHERE r.operationId = '8ff8696695aa73588ac454809741e2ea' RETURN n.id, r.duration, r.operation_Name, m.id ORDER BY r.duration DESC LIMIT 3
Full Context:
[{'n.id': 'ROOT', 'r.duration': 1868.0681, 'r.operation_Name': 'POST Timesheet/SubmitShiftEntry', 'm.id': 'v4-api'}, {'n.id': 'v4-api', 'r.duration': 1424.3322, 'r.operation_Name': 'POST Rules/Rules', 'm.id': 'rules-api'}, {'n.id': 'rules-api', 'r.duration': 787.6749, 'r.operation_Name': 'POST RulesData/GetPeopleDataForRules', 'm.id': 'people-api'}]

> Finished chain.
The root cause request ID 8ff8696695aa73588ac454809741e2ea is associated with the following chain of API calls:

1. **POST Timesheet/SubmitShiftEntry**
   - **Request ID**: 8ff8696695aa73588ac454809741e2ea
   - **Duration**: 1868.0681 ms
   - **Called API**: v4-api

2. **POST Rules/Rules**
   - **Request ID**: (inherited from v4-api)
   - **Duration**: 1424.3322 ms
   - **Called AP

In [156]:
pc_df = pd.read_csv('pc.csv')
pc_df.head(1)

,TimeGenerated [UTC],CounterName,InstanceName,avg_CounterValue
0,"12/7/2024, 3:00:00.000 pm",cpuUsageNanoCores,/subscriptions/a6b95c4d-8f36-4107-a2e9-272007e...,6469966.8


In [157]:
pc_df.iloc[0]['InstanceName']

'/subscriptions/a6b95c4d-8f36-4107-a2e9-272007e605c5/resourceGroups/cp-shared-perf/providers/Microsoft.ContainerService/managedClusters/shared-aks-perf/a7282838-5d83-46af-913c-a904ae789975/ama-logs'

In [158]:
def parseServiceName(x):
    splits = x.split('/')
    servicename = splits[-1]
    return servicename
def parseInstanceId(x):
    splits = x.split('/')
    instanceid = splits[-2]
    return instanceid
pc_df['ServiceName'] = pc_df['InstanceName'].apply(lambda x: parseServiceName(x))
pc_df['InstanceId'] = pc_df['InstanceName'].apply(lambda x: parseInstanceId(x))
# pc_df.drop(columns=['InstanceName'], inplace=True)
pc_df.rename(columns={'TimeGenerated [UTC]': 'timestamp'}, inplace=True)
pc_df.head(5)

,timestamp,CounterName,InstanceName,avg_CounterValue,ServiceName,InstanceId
0,"12/7/2024, 3:00:00.000 pm",cpuUsageNanoCores,/subscriptions/a6b95c4d-8f36-4107-a2e9-272007e...,6469966.8,ama-logs,a7282838-5d83-46af-913c-a904ae789975
1,"12/7/2024, 3:00:00.000 pm",cpuUsageNanoCores,/subscriptions/a6b95c4d-8f36-4107-a2e9-272007e...,1549.2,ama-logs-prometheus,a7282838-5d83-46af-913c-a904ae789975
2,"12/7/2024, 3:00:00.000 pm",cpuUsageNanoCores,/subscriptions/a6b95c4d-8f36-4107-a2e9-272007e...,646695.4,notification-api,dbcaa474-d058-4f26-ad2b-168a62be0dab
3,"12/7/2024, 3:00:00.000 pm",cpuUsageNanoCores,/subscriptions/a6b95c4d-8f36-4107-a2e9-272007e...,582428.4,address-api,878b658e-0fee-4b3d-984a-252a4b137fe5
4,"12/7/2024, 3:00:00.000 pm",cpuUsageNanoCores,/subscriptions/a6b95c4d-8f36-4107-a2e9-272007e...,613320.4,notification-api,731f7cef-8d41-4f99-b5cd-f101990ce9a7


In [159]:
# pivot the table to get the service names as columns
pc_df = pc_df.pivot(index='timestamp', columns='ServiceName', values='avg_CounterValue')

ValueError: Index contains duplicate entries, cannot reshape